In [4]:
import pandas as pd
import numpy as np
import re
import dask.dataframe as dd
from dask.multiprocessing import get
from tqdm import tqdm

In [163]:
def syn_gen(col_name):
    lst = [None]*25
    if col_name == 'nan':
        return ['nan']
    col_name = str(col_name)
    col_name = col_name.lower()
    name = col_name.split(',')[0]
    names = []
    names.append(name)                                           # As given
    
    name = re.subn('of|\sand|&', '', name, flags = re.IGNORECASE)[0]
    
    name = re.subn('\s+', ' ', name)[0]                          # Removing 'of' and 'and'
    names.append(name)
    names.append(''.join([word[0] for word in name.split()]).upper())
    
    if "'" in col_name:    
        names.append(''.join([word[0] for word in filter(None,re.split('\W+', name))]).upper())
    if col_name.count('.') > 1:
        names.append(''.join(re.split(r'\.\s?', col_name)[:-1]).upper())
        
        try:
            pattern = re.compile('\s?univ\S*\s?|\s?coll\S*\s?|\s?inst\S*\s?|\sres\S*\sfound\S*\s?')
            inst_type = pattern.findall(name)[0]
            names.append(names[-1]+inst_type)
        except:
            pass
        
    if ',' in col_name:
        location_full = [name+re.findall(',.*', col_name)[0][1:] for name in names]
        try:
            location_short = [name+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)]
            location_short.extend([name+'-'+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            location_short.extend([name+'/'+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            location_short.extend([name+' '+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            names.extend(location_short)
        except:
            pass
        names.extend(location_full)
    return (list(set(names))+lst)[:25]
syn_gen("L.K.KHOT ARTS COLLEGE, HEBBAL")

['LKH',
 'LAC hebbal',
 'LK',
 'LK hebbal',
 'LACH',
 'l.k.khot arts college',
 'LAC',
 'LK college',
 'LAC-H',
 'LAC H',
 'LK-H',
 'LK H',
 'l.k.khot arts college hebbal',
 'LK/H',
 'LAC/H',
 'LK college hebbal',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [164]:
data = pd.read_csv('../all_colleges_mereexams.csv')

/home/kunal_goyal/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [167]:
data.head()

,Unnamed: 0,cid,name,address_line1,address_line2,city,state,pin_code,area,constructed_area,...,no_of_separate_rooms_for_girls,solar_power_generation,connectivity_nkn,connectivity_nmeict,no_of_books,no_of_journals,campus_friendly,grievance_redressal_mechanism,vigilance_cell,opportunity_cell
0,0,C100,Manish Institute of Computer Studies,"S. K. Patel Campus, Kamana Char Rasta",Gandhinagar Ambaji State Highway,visnagar,Gujarat,384315,2.5,3467,...,1.0,False,False,False,8222.0,17.0,True,True,False,False
1,1,C10000,"SIDDARTHA COLLEGE OF EDUCATION, JAMKHANDI",BIJAPUR ROAD KATATE NAGAR JAMKHANDI,BIJAPUR ROAD KATATE NAGAR JAMKHANDI,jamkhandi,Karnataka,587301,3,1.77117999999999998,...,1.0,False,True,False,NaN,NaN,True,False,False,True
2,2,C10002,"VVS'S ARTS/COMM COLLEGE, BIJAPUR",STATION ROAD DARBAR HIGH SCHOOL CAMPUS,STATION ROAD,bijapur,Karnataka,586104,2,2000,...,NaN,False,False,False,5200.0,NaN,False,False,False,False
3,3,C10004,"GOVT. FIRST GRADE COLLEGE, KOKATANOOR",At Post Kokatanoor - 591 230 Tq: Athani,At Post Kokatanoor Tq: Athani,athani,Karnataka,591230,5,8500,...,1.0,False,False,False,6200.0,4.0,False,False,False,False
4,4,C10005,"S S S S'S COLLEGE OF BUSINESS ADMINISTRATION, ...","Nehru Nagar, Belguam","Nehru Nagar, Belguam",belguam,Karnataka,590010,3,13.2420000000000009,...,1.0,False,False,False,1600.0,5.0,False,False,False,False


In [169]:
def rm_brk(s):
    return re.sub('\(.*\)','',str(s))
clgs = pd.DataFrame(data.name)
clgs.head()

,name
0,Manish Institute of Computer Studies
1,"SIDDARTHA COLLEGE OF EDUCATION, JAMKHANDI"
2,"VVS'S ARTS/COMM COLLEGE, BIJAPUR"
3,"GOVT. FIRST GRADE COLLEGE, KOKATANOOR"
4,"S S S S'S COLLEGE OF BUSINESS ADMINISTRATION, ..."


In [170]:
names = pd.DataFrame(clgs['name'].apply(syn_gen))

In [171]:
names.head()

,name
0,"[MICS, manish institute of computer studies, m..."
1,"[SCE/J, siddartha college education, SCE jamk..."
2,"[VAC, VSACCB, VSACC bijapur, VAC-B, VAC B, VS..."
3,"[GFGC-K, GFGC/K, GFGC K, GFGC kokatanoor, gov..."
4,[s s s s's college business administration be...


In [172]:
names.iloc[0]['name']

['MICS',
 'manish institute of computer studies',
 'manish institute computer studies',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [173]:
len(names.iloc[4]['name'])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [174]:
names.name.values.tolist()
names[list(range(25))] = pd.DataFrame(names.name.values.tolist(), index= names.index)

In [178]:
names['name']=data['name']

In [179]:
def comma(s):
    if str(s) != str(None):
        return '"'+s+'"'
    return None

In [180]:
names = names.apply(comma)

In [181]:
names[names[24].notnull()]

,name,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24


In [182]:
names = names.drop([24],axis=1)

In [183]:
names.to_csv('clg_entities.csv',header=None,index=None)

In [157]:
rm_brk('kuna(kd)l')

'kunal'

In [166]:
data['name'] = data['name'].apply(rm_brk)

In [185]:
data.to_csv('all_clgs.csv')

In [186]:
names.head()

,name,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,"""Manish Institute of Computer Studies""","""MICS""","""manish institute of computer studies""","""manish institute computer studies""",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"""SIDDARTHA COLLEGE OF EDUCATION, JAMKHANDI""","""SCE/J""","""siddartha college education""","""SCE jamkhandi""","""SCE-J""","""siddartha college of education""","""siddartha college education jamkhandi""","""siddartha college of education jamkhandi""","""SCE""","""SCE J""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""VVS'S ARTS/COMM COLLEGE, BIJAPUR""","""VAC""","""VSACCB""","""VSACC bijapur""","""VAC-B""","""VAC B""","""VSACC/B""","""VSACC B""","""vvs's arts/comm college bijapur""","""VAC bijapur""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""GOVT. FIRST GRADE COLLEGE, KOKATANOOR""","""GFGC-K""","""GFGC/K""","""GFGC K""","""GFGC kokatanoor""","""govt. first grade college kokatanoor""","""govt. first grade college""","""GFGC""","""GFGCK""",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""S S S S'S COLLEGE OF BUSINESS ADMINISTRATION,...","""s s s s's college business administration be...","""SSSSCBA B""","""s s s s's college of business administration ...","""SSSSCBA/B""","""SSSSSCBA/B""","""SSSSCBA belgaum""","""SSSSCBA-B""","""SSSSSCBA""","""s s s s's college of business administration""",...,"""s s s s's college business administration""","""SSSSSCBA-B""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


college_entity.ipynb               merger.ipynb
college_front_page_scrapper.ipynb  scrapper2.ipynb
entity_generator.ipynb             temp_mapper.ipynb
entity_synonym_generator.ipynb     univ_names
mapper.ipynb


In [5]:
data = pd.read_csv('CollegeName_entity.csv')

In [10]:
data.iloc[:15000].to_csv('CollgeName1',header=None,index=None)

In [11]:
data.iloc[15000:].to_csv('CollegeName2',header=None,index=None)